In [33]:
Select top (5)
    OrderNum,
    OrderLine,
    OrderNumLine
From [ERPDB].[etl].[vwOrderDetail]
WHERE Company = 'Solar' and OrderNum Like '2170022%'

(5 rows affected)

Total execution time: 00:00:00.070

OrderNum,OrderLine,OrderNumLine
2170022,1,2170022-1
2170022,2,2170022-2
2170022,3,2170022-3
2170022,4,2170022-4
2170022,5,2170022-5


In [31]:
Select top (5)
    JobNum,
    OrderNum,
    OrderLine,
    OrderNumLine
From [ERPDB].[etl].[vwJobMtlChange]
WHERE Company = 'Solar' and JobNum Like '2170022%'

(5 rows affected)

Total execution time: 00:00:06.208

JobNum,OrderNum,OrderLine,OrderNumLine
2170022-13-1,2170022,13,2170022-13
2170022-2-1,2170022,2,2170022-2
2170022-4-1,2170022,4,2170022-4
2170022-4-1,2170022,4,2170022-4
2170022-5-1,2170022,5,2170022-5


In [32]:
Select  top (5)
    JobNum,
    OrderNum,
    OrderLine,
    OrderNumLine
From [ERPDB].[etl].[vwPartTran]
WHERE Company = 'Solar' and JobNum Like '2170022%'

(5 rows affected)

Total execution time: 00:00:00.066

JobNum,OrderNum,OrderLine,OrderNumLine
2170022-13-1,2170022,13,2170022-13
2170022-2-1,2170022,2,2170022-2
2170022-2-1,2170022,2,2170022-2
2170022-2-1,2170022,2,2170022-2
2170022-2-1,2170022,2,2170022-2


In [30]:
 Select top (5)
     JobNum,
    OrderNum,
    OrderLine,
    OrderNumLine
From [ERPDB].[etl].[vwLaborDtl]
WHERE Company = 'Solar' and JobNum Like '2170022%'

(5 rows affected)

Total execution time: 00:00:00.089

JobNum,OrderNum,OrderLine,OrderNumLine
2170022-1-1,2170022,1,2170022-1
2170022-1-1,2170022,1,2170022-1
2170022-1-1,2170022,1,2170022-1
2170022-1-1,2170022,1,2170022-1
2170022-1-1,2170022,1,2170022-1


In [15]:
---Final sample code for view creation

SELECT jm.*, cl.DateStampedOn, cc.Calculated_ChangeDate,
 (case when
            cc.Calculated_ChangeDate >= cl.DateStampedOn
            then cc.Calculated_ChangeDate
            else 
            cl.DateStampedON
            end) as Latest_ChangeDate
,Dateadd(second, 00, cast(
    (case when
            cc.Calculated_ChangeDate >= cl.DateStampedOn
            then cc.Calculated_ChangeDate
            else 
            cl.DateStampedON
            end)as Datetime)) as Latest_ChangeDateTime
FROM
(
    SELECT JobMtl.*,
    REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 1)) as OrderNum,
    REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 2)) as OrderLine,
    (Case
      When
        REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 2)) is Null
      Then
        REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 1))
      Else
        CONCAT(REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 1))
        ,'-'
        ,REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 2))
      )
      end) as OrderNumLine,
    CONCAT(JobNum,'~',AssemblySeq,'~',MtlSeq) as ForeignKey2
    FROM [ERPDB].[Erp].[JobMtl]
) jm

-- Get the DateStampedOn from the ChangeLog
LEFT JOIN
(
Select *
From
    (Select DateStampedOn,
            Key1,
            Key2,
            LogText,
            ROW_NUMBER() OVER(Partition by Key2 Order by DateStampedON Desc) rn
        From [ERPDB].[Ice].[ChgLog] 
    ) a
    Where rn=1 
) cl
ON jm.JobNum=cl.Key1 and jm.ForeignKey2=cl.Key2


--get the calculated change date from PartTran and Job Head Tables 
Left Join
(
Select  *
From 
    (Select 
        f.Company,
        f.JobNum,
        f.AssemblySeq,
        f.JobSeq,
        f.PartNum,
        f.TranDate,
        f.CreateDate,
        f.CloseDate,
        f.ForeignKey2,
        f.Calculated_ChangeDate,
        ROW_NUMBER() OVER(PARTITION By ForeignKey2 ORDER BY f.Calculated_ChangeDate DESC) rn
    FROM
        (SELECT 
            pt.Company as Company, 
            pt.JobNum as JobNum,
            pt.AssemblySeq as AssemblySeq,
            pt.JobSeq as JobSeq,
            pt.TranDate as TranDate,
            pt.PartNum as PartNum,
            jh.CreateDate as CreateDate,
            jh.ClosedDate as CloseDate,
            pt.ForeignKey2 as ForeignKey2,
            (case when
            pt.TranDate >= jh.CreateDate
            then
            pt.TranDate
            when
            jh.CreateDate >= jh.ClosedDate
            then
            jh.CreateDate
            else
            jh.ClosedDate
            end) as Calculated_ChangeDate
                FROM
                (SELECT
                    Company,
                    JobNum,
                    AssemblySeq,
                    JobSeq,
                    TranDate,
                    PartNum,
                    CONCAT(JobNum,'~',AssemblySeq,'~',JobSeq) as ForeignKey2
                    FROM [ERPDB].[Erp].[PartTran]
                ) as pt
                Left Join
                (SELECT 
                    Company, 
                    ClosedDate, 
                    JobNum,
                    CreateDate 
                    FROM [ERPDB].[Erp].[JobHead]
                ) as jh
                    on pt.Company = jh.Company AND pt.JobNum = jh.JobNum
                WHERE pt.Jobnum like '2170022%' and pt.Company = 'Solar'
        ) f 
    ) b
    where rn = 1
) cc
on jm.ForeignKey2 = cc.ForeignKey2

WHERE jm.Jobnum like '2170022%' and jm.Company = 'Solar'
Order By Latest_ChangeDate DESC

(30 rows affected)

Total execution time: 00:00:04.039

Company,JobComplete,IssuedComplete,JobNum,AssemblySeq,MtlSeq,PartNum,Description,QtyPer,RequiredQty,IUM,LeadTime,RelatedOperation,MtlBurRate,EstMtlBurUnitCost,EstUnitCost,IssuedQty,TotalCost,MtlBurCost,ReqDate,WarehouseCode,SalvagePartNum,SalvageDescription,SalvageQtyPer,SalvageUM,SalvageMtlBurRate,SalvageUnitCredit,SalvageEstMtlBurUnitCredit,SalvageQtyToDate,SalvageCredit,SalvageMtlBurCredit,MfgComment,VendorNum,PurPoint,BuyIt,Ordered,PurComment,BackFlush,EstScrap,EstScrapType,FixedQty,FindNum,RevisionNum,SndAlrtCmpl,RcvInspectionReq,Plant,Direct,MaterialMtlCost,MaterialLabCost,MaterialSubCost,MaterialBurCost,SalvageMtlCredit,SalvageLbrCredit,SalvageBurCredit,SalvageSubCredit,APSAddResType,CallNum,CallLine,ProdCode,UnitPrice,BillableUnitPrice,DocBillableUnitPrice,ResReasonCode,PricePerCode,Billable,ShippedQty,DocUnitPrice,QtyStagedToDate,AddedMtl,MiscCharge,MiscCode,SCMiscCode,RFQNeeded,RFQVendQuotes,RFQNum,RFQLine,RFQStat,AnalysisCode,GlbRFQ,WhseAllocFlag,WIReqDate,Rpt1BillableUnitPrice,Rpt2BillableUnitPrice,Rpt3BillableUnitPrice,Rpt1UnitPrice,Rpt2UnitPrice,Rpt3UnitPrice,BaseRequiredQty,BaseUOM,Weight,WeightUOM,ReqRefDes,BasePartNum,BaseRevisionNum,SelectForPicking,StagingWarehouseCode,StagingBinNum,PickError,EstMtlUnitCost,EstLbrUnitCost,EstBurUnitCost,EstSubUnitCost,SalvageEstMtlUnitCredit,SalvageEstLbrUnitCredit,SalvageEstBurUnitCredit,SalvageEstSubUnitCredit,LoanedQty,BorrowedQty,ReassignSNAsm,GeneralPlanInfo,EstStdDescription,PricingUOM,RemovedFromPlan,IsPOCostingMaintained,EstStdType,POCostingFactor,PlannedQtyPerUnit,POCostingDirection,POCostingUnitVal,GroupSeq,SysRevID,SysRowID,OrigStructTag,OrigGroupSeq,ShowStatusIcon,ContractID,LinkToContract,StagingLotNum,PCLinkRemoved,ExternalMESSyncRequired,ExternalMESLastSync,LocationView,AttributeSetID,PlanningNumberOfPieces,SalvageAttributeSetID,SalvagePlanningNumberOfPieces,SalvagePlanningAttributeSetID,OrderNum,OrderLine,OrderNumLine,ForeignKey2,DateStampedOn,Calculated_ChangeDate,Latest_ChangeDate,Latest_ChangeDateTime
SOLAR,0,1,2170022-5-1,0,40,04000,FC Contracting,1.00000000,1.00000000,EA,0,0,0.00000,0.00000,1110419.00000,1.00000000,1139319.000,0.000,2021-03-12,,,,0.00000000,,0.00000,0.00000,0.00000,0.00000000,0.000,0.000,,0,,0,0,,0,0.00,%,1,,,0,0,MfgSys,0,1139319.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,M,0,0,,0.00000,0.00000,0.00000,,E,0,0.00000000,0.00000,0.00000000,0,1,MISC,,0,0,0,0,,,0,0,2021-03-12,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000000,EA,0.00000,,0,,,0,,,,1110419.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00,0.00,0,,,,0,0,0,0.00000,0.00000,0,0.00000,0,0x0000000328216124,08b6e033-4f7f-4291-8b07-3a1608dbd811,,0,Complete,,0,,0,1,NULL,0,0,0,0,0,0,2170022,5,2170022-5,2170022-5-1~0~40,2022-05-31,2022-07-18,2022-07-18,2022-07-18 00:00:00.000
SOLAR,0,1,2170022-5-1,0,50,05000,William Scotsman Trailer,1.00000000,1.00000000,EA,0,0,0.00000,0.00000,10500.00000,1.00000000,6904.770,0.000,2021-03-12,,,,0.00000000,,0.00000,0.00000,0.00000,0.00000000,0.000,0.000,,0,,0,0,,0,0.00,%,1,,,0,0,MfgSys,0,6904.770,0.000,0.000,0.000,0.000,0.000,0.000,0.000,M,0,0,,0.00000,0.00000,0.00000,,E,0,0.00000000,0.00000,0.00000000,0,1,MISC,,0,0,0,0,,,0,0,2021-03-12,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000000,EA,0.00000,,0,,,0,,,,10500.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00,0.00,0,,,,0,0,0,0.00000,0.00000,0,0.00000,0,0x00000003282161FB,712f14d3-482a-459f-b55c-4d3c26871ecf,,0,Complete,,0,,0,1,NULL,0,0,0,0,0,0,2170022,5,2170022-5,2170022-5-1~0~50,2022-05-31,2022-07-18,2022-07-18,2022-07-18 00:00:00.000
SOLAR,0,1,2170022-2-1,0,10,MFG MATERIAL,MFG MATERIAL,1.00000000,1.00000000,EA,0,0,0.00000,0.00000,0.00000,1.00000000,3875930.000,0.000,2021-03-12,,,,0.00000000,,0.00000,0.00000,0.00000,0.00000000,0.000,0.000,,0,,0,0,,0,0.00,%,1,,A,0,0,MfgSys,0,3875930.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,M,0,0,MFG,0.00000,0.00000,0.00000,,E,0,0.00000000,0.00000,0.00000000,0,1,MISC,,0,0,0,0,,,0,0,2021-03-12,0.00000,0.00000,0.000

In [9]:
SELECT 
TABLE_CATALOG,
TABLE_SCHEMA,
TABLE_NAME, 
COLUMN_NAME, 
DATA_TYPE 
FROM INFORMATION_SCHEMA.COLUMNS
where TABLE_NAME = 'vwOrderDetail' 
and COLUMN_NAME = 'OrderLine'

(1 row affected)

Total execution time: 00:00:00.058

TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE
ERPDB,etl,vwOrderDetail,OrderLine,int


In [7]:
Select Top (10) *
  FROM [ERPDB].[etl].[vwOrderDetail]
  Where Company = 'Solar'
  Order BY ChangeDate DESC

(10 rows affected)

Total execution time: 00:00:00.252

VoidLine,OpenLine,Company,OrderNum,OrderLine,LineType,PartNum,LineDesc,Reference,IUM,RevisionNum,POLine,Commissionable,DiscountPercent,UnitPrice,DocUnitPrice,OrderQty,Discount,DocDiscount,RequestDate,ProdCode,XPartNum,XRevisionNum,PricePerCode,OrderComment,ShipComment,InvoiceComment,PickListComment,TaxCatID,AdvanceBillBal,DocAdvanceBillBal,QuoteNum,QuoteLine,TMBilling,OrigWhyNoTax,NeedByDate,CustNum,Rework,RMANum,RMALine,ProjectID,ContractNum,ContractCode,BasePartNum,Warranty,WarrantyCode,MaterialDuration,LaborDuration,MiscDuration,MaterialMod,LaborMod,WarrantyComment,Onsite,MatCovered,LabCovered,MiscCovered,SalesUM,SellingFactor,SellingQuantity,SalesCatID,ShipLineComplete,CumeQty,CumeDate,MktgCampaignID,MktgEvntSeq,LockQty,Linked,ICPONum,ICPOLine,ExtCompany,LastConfigDate,LastConfigTime,LastConfigUserID,ConfigUnitPrice,ConfigBaseUnitPrice,PriceListCode,BreakListCode,PricingQty,LockPrice,ListPrice,DocListPrice,OrdBasedPrice,DocOrdBasedPrice,PriceGroupCode,OverridePriceList,BaseRevisionNum,PricingValue,DisplaySeq,KitParentLine,KitAllowUpdate,KitShipComplete,KitBackFlush,KitPrintCompsPS,KitPrintCompsInv,KitPricing,KitQtyPer,SellingFactorDirection,RepRate1,RepRate2,RepRate3,RepRate4,RepRate5,RepSplit1,RepSplit2,RepSplit3,RepSplit4,RepSplit5,DemandContractLine,CreateNewJob,DoNotShipBeforeDate,GetDtls,DoNotShipAfterDate,SchedJob,RelJob,EnableCreateNewJob,EnableGetDtls,EnableSchedJob,EnableRelJob,CounterSaleWarehouse,CounterSaleBinNum,CounterSaleLotNum,CounterSaleDimCode,DemandDtlRejected,KitFlag,KitsLoaded,DemandContractNum,DemandHeadSeq,DemandDtlSeq,ChangedBy,ChangeDate,ChangeTime,ChangeDateTime,ReverseCharge,TotalReleases,Rpt1UnitPrice,Rpt2UnitPrice,Rpt3UnitPrice,Rpt1Discount,Rpt2Discount,Rpt3Discount,Rpt1AdvanceBillBal,Rpt2AdvanceBillBal,Rpt3AdvanceBillBal,Rpt1ListPrice,Rpt2ListPrice,Rpt3ListPrice,Rpt1OrdBasedPrice,Rpt2OrdBasedPrice,Rpt3OrdBasedPrice,ExtPriceDtl,DocExtPriceDtl,Rpt1ExtPriceDtl,Rpt2ExtPriceDtl,Rpt3ExtPriceDtl,LineStatus,InUnitPrice,DocInUnitPrice,InDiscount,DocInDiscount,InListPrice,DocInListPrice,InOrdBasedPrice,DocInOrdBasedPrice,Rpt1InUnitPrice,Rpt2InUnitPrice,Rpt3InUnitPrice,Rpt1InDiscount,Rpt2InDiscount,Rpt3InDiscount,Rpt1InListPrice,Rpt2InListPrice,Rpt3InListPrice,Rpt1InOrdBasedPrice,Rpt2InOrdBasedPrice,Rpt3InOrdBasedPrice,InExtPriceDtl,DocInExtPriceDtl,Rpt1InExtPriceDtl,Rpt2InExtPriceDtl,Rpt3InExtPriceDtl,OldOurOpenQty,OldSellingOpenQty,OldOpenValue,OldProdCode,PrevSellQty,PrevPartNum,PrevXPartNum,KitCompOrigSeq,KitCompOrigPart,SmartStringProcessed,SmartString,RenewalNbr,DiscBreakListCode,DiscListPrice,LockDisc,OverrideDiscPriceList,GroupSeq,ECCOrderNum,ECCOrderLine,DupOnJobCrt,UndersPct,Overs,Unders,OversUnitPrice,PlanUserID,PlanGUID,MOMsourceType,MOMsourceEst,DefaultOversPricing,ECCPlant,ECCQuoteNum,ECCQuoteLine,SysRevID,SysRowID,MfgJobType,ProFormaInvComment,CreateJob,ContractID,LinkToContract,DocInAdvanceBillBal,InAdvanceBillBal,Rpt1InAdvanceBillBal,Rpt2InAdvanceBillBal,Rpt3InAdvanceBillBal,PCLinkRemoved,CommodityCode,MSRP,DocMSRP,Rpt1MSRP,Rpt2MSRP,Rpt3MSRP,EndCustomerPrice,DocEndCustomerPrice,Rpt1EndCustomerPrice,Rpt2EndCustomerPrice,Rpt3EndCustomerPrice,PromotionalPrice,DocPromotionalPrice,Rpt1PromotionalPrice,Rpt2PromotionalPrice,Rpt3PromotionalPrice,OrderLineStatusCode,AttributeSetID,ForeignSysRowID,UD_SysRevID,DoNotBook_c,EstUnitCost_c,OriginalEstimate_c
0,1,SOLAR,2230025,4,PART,FREIGHT,FREIGHT,,EA,,,1,0.00,0.00000,0.00000,1.00000000,0.000,0.000,2022-03-17,03FRT,,,E,,,,,,0.00000,0.00000,0,0,0,,2022-03-17,587,0,0,0,2230025,0,,,1,,0,0,0,,,,0,0,0,0,EA,1.00000000,1.00000000,GM,0,0.00000000,NULL,DEFAULT,1,0,0,0,0,,NULL,0,,0.00000,0.00000,,,0.00000000,1,0.00000,0.00000,0.00000,0.00000,,0,,0.000,4.000,4,0,0,0,0,0,,0.00000000,D,0.00,0.00,0.00,0.00,0.00,100,0,0,0,0,0,1,NULL,0,NULL,0,0,1,1,1,1,,,,,0,,0,0,0,0,Adamw,2022-07-18,64059,2022-07-18 17:47:39.000,0,1,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000,0.000,0.000,0.000,0.000,OP

In [14]:
 Select Top (1000)
    Company,
    OrderNum,
    OrderLine,
    ChangeDate,
    (Case
      When
        Orderline is Null
      Then
        CAST(OrderNum AS VARCHAR(15))
      Else
        CONCAT(CAST(OrderNum AS VARCHAR(15))
        ,'-'
        ,CAST(OrderLine AS VARCHAR(15))
        )
      end) as OrderNumLine
  FROM [ERPDB].[etl].[vwOrderDetail]
  Where Company = 'Solar'
  Order BY ChangeDate DESC

(1000 rows affected)

Total execution time: 00:00:00.099

Company,OrderNum,OrderLine,ChangeDate,OrderNumLine
SOLAR,2230025,4,2022-07-18,2230025-4
SOLAR,2230025,1,2022-07-18,2230025-1
SOLAR,2174034,10,2022-07-18,2174034-10
SOLAR,2174034,8,2022-07-18,2174034-8
SOLAR,2174034,4,2022-07-18,2174034-4
SOLAR,2174034,2,2022-07-18,2174034-2
SOLAR,2296044,11,2022-07-18,2296044-11
SOLAR,2296044,10,2022-07-18,2296044-10
SOLAR,2296044,9,2022-07-18,2296044-9
SOLAR,2296044,8,2022-07-18,2296044-8


In [5]:
 Select Top (1000)
    Company,
    [JobNum],
    PayrollDate,
    REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 1)) as OrderNum,
    REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 2)) as OrderLine,
    (Case
      When
        REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 2)) is Null
      Then
        REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 1))
      Else
        CONCAT(REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 1))
        ,'-'
        ,REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 2))
      )
      end) as OrderNumLine,
    Left(JobNum, CHARINDEX('-', JobNum) + 2) as OrderNumLine2
  FROM [ERPDB].[etl].[vwLaborDtl]
  Where (JobNum IS NOT NULL AND JobNum != ' ') and Company = 'Solar'
  Order BY PayrollDate DESC

(1000 rows affected)

Total execution time: 00:00:00.150

Company,JobNum,PayrollDate,OrderNum,OrderLine,OrderNumLine,OrderNumLine2
SOLAR,2170070-1-1,2022-07-18,2170070,1,2170070-1,2170070-1-
SOLAR,2230055-1-1,2022-07-18,2230055,1,2230055-1,2230055-1-
SOLAR,2230071-1-1,2022-07-18,2230071,1,2230071-1,2230071-1-
SOLAR,2174028-1-1,2022-07-18,2174028,1,2174028-1,2174028-1-
SOLAR,2170070-1-1,2022-07-18,2170070,1,2170070-1,2170070-1-
SOLAR,2270005-1-1,2022-07-18,2270005,1,2270005-1,2270005-1-
SOLAR,2270005-1-1,2022-07-18,2270005,1,2270005-1,2270005-1-
SOLAR,2270005-1-1,2022-07-18,2270005,1,2270005-1,2270005-1-
SOLAR,2170083-1-1,2022-07-18,2170083,1,2170083-1,2170083-1-
SOLAR,2170083-1-1,2022-07-18,2170083,1,2170083-1,2170083-1-


In [4]:
 Select Top (1000)
    Company,
    [JobNum],
    TranDate,
    REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 1)) as OrderNum,
    REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 2)) as OrderLine,
    (Case
      When
        REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 2)) is Null
      Then
        REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 1))
      Else
        CONCAT(REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 1))
        ,'-'
        ,REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 2))
      )
      end) as OrderNumLine,
    Left(JobNum, CHARINDEX('-', JobNum) + 2) as OrderNumLine2
  FROM [ERPDB].[etl].[vwPartTran]
  Where (JobNum IS NOT NULL AND JobNum != ' ') and Company = 'Solar'
  Order BY TranDate DESC

(1000 rows affected)

Total execution time: 00:00:00.240

Company,JobNum,TranDate,OrderNum,OrderLine,OrderNumLine,OrderNumLine2
SOLAR,4000006-4-1,2022-08-01,4000006,4,4000006-4,4000006-4-
SOLAR,4000042-5-1,2022-08-01,4000042,5,4000042-5,4000042-5-
SOLAR,4000006-5-1,2022-08-01,4000006,5,4000006-5,4000006-5-
SOLAR,2239997-8-1,2022-07-18,2239997,8,2239997-8,2239997-8-
SOLAR,2274004-5-1,2022-07-18,2274004,5,2274004-5,2274004-5-
SOLAR,2170023-5-1,2022-07-18,2170023,5,2170023-5,2170023-5-
SOLAR,2170023-5-1,2022-07-18,2170023,5,2170023-5,2170023-5-
SOLAR,2179999-5-1,2022-07-18,2179999,5,2179999-5,2179999-5-
SOLAR,2170022-5-1,2022-07-18,2170022,5,2170022-5,2170022-5-
SOLAR,2230025-5-1,2022-07-18,2230025,5,2230025-5,2230025-5-
